In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hypertrophic_Cardiomyopathy"
cohort = "GSE36961"

# Input paths
in_trait_dir = "../../input/GEO/Hypertrophic_Cardiomyopathy"
in_cohort_dir = "../../input/GEO/Hypertrophic_Cardiomyopathy/GSE36961"

# Output paths
out_data_file = "../../output/preprocess/Hypertrophic_Cardiomyopathy/GSE36961.csv"
out_gene_data_file = "../../output/preprocess/Hypertrophic_Cardiomyopathy/gene_data/GSE36961.csv"
out_clinical_data_file = "../../output/preprocess/Hypertrophic_Cardiomyopathy/clinical_data/GSE36961.csv"
json_path = "../../output/preprocess/Hypertrophic_Cardiomyopathy/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome Profiling of Surgical Myectomy Tissue from Patients with Hypertrophic Cardiomyopathy"
!Series_summary	"Using a high-throughput gene expression profiling technology, we have been able to develop new hypotheses regarding the molecular pathogenic mechanisms of human hypertrophic cardiomyopathy (HCM). It is hoped that these hypotheses, among others generated by this data, will fuel future research endeavors that will uncover novel biomarkers, prognostic indicators, and therapeutic targets to improve our ability to diagnose, counsel, and treat patients with this highly heterogeneous and potentially life-threatening condition."
!Series_overall_design	"Case-control study comparing the messenger RNA transcriptome of cardiac tissues from patients with hypertrophic cardiomyopathy to the transcriptome of control donor cardiac tissues."
Sample Characteristics Dictionary:
{0: ['Sex: male', 'Sex: female'], 1: ['age (yrs): 9', 'age (yrs): 10', 'age

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Check if gene expression data is available
# The background information mentions "messenger RNA transcriptome" comparing HCM to control samples
# This suggests gene expression data is available.
is_gene_available = True

# Define conversion functions for clinical variables
def convert_trait(value):
    """Convert trait value to binary (0=control, 1=HCM)"""
    if pd.isna(value):
        return None
        
    value = value.lower() if isinstance(value, str) else str(value).lower()
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "hypertrophic cardiomyopathy" in value or "hcm" in value or "case" in value:
        return 1
    elif "control" in value:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous"""
    if pd.isna(value):
        return None
        
    value = str(value)
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Extract numeric age
    match = re.search(r'(\d+)', value)
    if match:
        return int(match.group(1))
    else:
        return None

def convert_gender(value):
    """Convert gender value to binary (0=female, 1=male)"""
    if pd.isna(value):
        return None
        
    value = value.lower() if isinstance(value, str) else str(value).lower()
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# Identify rows with trait, age, and gender information
trait_row = None
age_row = None
gender_row = None

# Looking at the sample characteristics dictionary
# Row 0 contains gender information ("Sex: male", "Sex: female")
gender_row = 0

# Row 1 contains age information with format "age (yrs): XX"
age_row = 1

# Row 3 contains disease state information about HCM vs control
trait_row = 3

# Determine trait data availability
is_trait_available = trait_row is not None

# Save metadata with initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if trait_row is not None:
    # Use geo_select_clinical_features to extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical dataframe
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM907203': [1.0, 9.0, 1.0], 'GSM907204': [1.0, 10.0, 1.0], 'GSM907205': [1.0, 10.0, 0.0], 'GSM907206': [1.0, 11.0, 1.0], 'GSM907207': [1.0, 13.0, 0.0], 'GSM907208': [1.0, 14.0, 1.0], 'GSM907209': [1.0, 15.0, 1.0], 'GSM907210': [1.0, 15.0, 0.0], 'GSM907211': [1.0, 15.0, 1.0], 'GSM907212': [1.0, 15.0, 1.0], 'GSM907213': [1.0, 16.0, 0.0], 'GSM907214': [1.0, 16.0, 1.0], 'GSM907215': [1.0, 17.0, 0.0], 'GSM907216': [1.0, 19.0, 1.0], 'GSM907217': [1.0, 19.0, 1.0], 'GSM907218': [1.0, 20.0, 0.0], 'GSM907219': [1.0, 23.0, 1.0], 'GSM907220': [1.0, 23.0, 0.0], 'GSM907221': [1.0, 26.0, 1.0], 'GSM907222': [1.0, 27.0, 1.0], 'GSM907223': [1.0, 28.0, 1.0], 'GSM907224': [1.0, 30.0, 1.0], 'GSM907225': [1.0, 30.0, 0.0], 'GSM907226': [1.0, 30.0, 0.0], 'GSM907227': [1.0, 31.0, 1.0], 'GSM907228': [1.0, 32.0, 0.0], 'GSM907229': [1.0, 32.0, 0.0], 'GSM907230': [1.0, 33.0, 0.0], 'GSM907231': [1.0, 35.0, 0.0], 'GSM907232': [1.0, 35.0, 0.0], 'GSM907233': [1.0, 37.0, 0.0], 'GSM907234': [1.

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 37846 rows
First 20 gene IDs:
Index(['7A5', 'A1BG', 'A1CF', 'A26A1', 'A26B1', 'A26C1B', 'A26C3', 'A2BP1',
       'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS',
       'AACSL', 'AADAC', 'AADACL1', 'AADACL2'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers in the gene expression data

# The gene identifiers in the dataset appear to be standard human gene symbols.
# Examples like A1BG, AAAS, AACS are recognized human gene symbols.
# These are official HUGO Gene Nomenclature Committee (HGNC) symbols
# and do not require additional mapping to be used in analysis.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols and extract from step 3 and 6
# Load the gene expression data (already loaded from Step 6)
print(f"Gene data shape before normalization: {gene_data.shape}")

try:
    # Normalize gene symbols using the NCBI Gene database information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    if normalized_gene_data.empty:
        print("Normalization resulted in empty dataframe. Using original gene data instead.")
        normalized_gene_data = gene_data
    
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data to the output file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}. Using original gene data instead.")
    normalized_gene_data = gene_data
    # Save the original gene data if normalization fails
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)

# 2. Link clinical and genetic data
# Use the trait_row identified in Step 2 (trait_row = 1) to extract trait data
is_trait_available = trait_row is not None

if is_trait_available:
    # Extract clinical features using the function and conversion methods from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Create a minimal dataframe with just the trait column
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No trait data available, creating minimal dataframe for validation.")

# 3. Handle missing values in the linked data
if is_trait_available:
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After missing value handling, linked data shape: {linked_data.shape}")

# 4. Determine whether trait and demographic features are biased
if is_trait_available and not linked_data.empty and len(linked_data.columns) > 1:
    print("\nEvaluating feature bias...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Trait bias evaluation result: {is_biased}")
else:
    is_biased = False
    print("Skipping bias evaluation due to insufficient data.")

# 5. Final validation and save metadata
note = ""
if not is_trait_available:
    note = f"Dataset contains gene expression data but no {trait} measurements."
elif is_biased:
    note = f"Dataset contains {trait} data but its distribution is severely biased."

# Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset is not usable for {trait} association studies. Data not saved.")

Gene data shape before normalization: (37846, 145)
Gene data shape after normalization: (18660, 145)


Normalized gene data saved to ../../output/preprocess/Hypertrophic_Cardiomyopathy/gene_data/GSE36961.csv
Clinical data saved to ../../output/preprocess/Hypertrophic_Cardiomyopathy/clinical_data/GSE36961.csv
Linked data shape: (145, 18663)

Handling missing values...


After missing value handling, linked data shape: (142, 18663)

Evaluating feature bias...
For the feature 'Hypertrophic_Cardiomyopathy', the least common label is '0.0' with 36 occurrences. This represents 25.35% of the dataset.
The distribution of the feature 'Hypertrophic_Cardiomyopathy' in this dataset is fine.

Quartiles for 'Age':
  25%: 30.0
  50% (Median): 47.0
  75%: 58.0
Min: 4.0
Max: 78.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 69 occurrences. This represents 48.59% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Trait bias evaluation result: False
A new JSON file was created at: ../../output/preprocess/Hypertrophic_Cardiomyopathy/cohort_info.json

Dataset usability: True


Linked data saved to ../../output/preprocess/Hypertrophic_Cardiomyopathy/GSE36961.csv
